In [4]:
discrim_dnn = ['300x150x70x35x1', '400x200x50x1', '512x256x1']
gener_dnn = ['256x300', '128x300']
index = ['8E-3', '5E-3']

for j in range(len(discrim_dnn)):
    for l in range(len(gener_dnn)):
        for m in range(len(index)):
            f = open('../adversarial/noiseAdv_hingemaxbatch_DNN'+str(j)+str(l)+str(m)+'.job','w')
            f.write('#!/bin/bash\n')
            f.write('#$ -S /bin/bash\n')
            f.write('#$ -M jontsai@uchicago.edu\n')
            f.write('#$ -N noiseAdv_hingemaxbatchDNN'+str(j)+str(l)+str(m)+'\n')
            f.write('#$ -m beasn\n')
            f.write('#$ -o noiseAdv_hingemaxbatchDNN'+str(j)+str(l)+str(m)+'.out\n')
            f.write('#$ -e noiseAdv_hingemaxbatchDNN'+str(j)+str(l)+str(m)+'.err\n')
            f.write('#$ -r n\n')
            f.write('#$ -cwd\n')
            f.write('SETTING1="300"\n')
            f.write('SETTING2="300"\n')
            f.write('SETTING3="150"\n')
            f.write('SETTING4="'+gener_dnn[l]+'"\n')
            f.write('SETTING5="0"\n')
            f.write('SETTING6="0.0005"\n')
            f.write('SETTING7="0.0005"\n')            
            f.write('SETTING8="1.0"\n')
            f.write('SETTING9="sequence"\n')
            f.write('SETTING10="default"\n')
            f.write('SETTING11="DNN"\n')
            f.write('SETTING12="hinge"\n')
            f.write('SETTING13="normal"\n')
            f.write('SETTING14="'+discrim_dnn[j]+'"\n')
            f.write('SETTING15="1"\n')
            f.write('SETTING16="1"\n')
            f.write('SETTING17="'+index[m]+'"\n')
            f.write('SETTING18="noiseAdv_hingemaxbatchDNN'+str(j)+str(l)+str(m)+'"\n')
            f.write('export OMP_NUM_THREADS=1\n')
            f.write('export OPENBLAS_NUM_THREADS=1\n')
            f.write('echo "Start - `date`"\n')
            f.write('/home-nfs/jontsai/anaconda/bin/python HierBLSTM_hingeNoiseAdv.py \\\n')
            f.write('$SETTING1 $SETTING2 $SETTING3 $SETTING4 $SETTING5 $SETTING6 $SETTING7 $SETTING8 $SETTING9 $SETTING10 $SETTING11 $SETTING12 $SETTING13 $SETTING14 $SETTING15 $SETTING16 $SETTING17\n')
            f.write('echo "End - `date`"\n')
            f.close()

In [13]:
import numpy as np

best_val_acc_ls = np.zeros((3,2,2))

best_test_acc_ls = np.zeros((3,2,2))

for i in range(3):
    for j in range(2):
        for k in range(2):
            f = open('../adversarial/noiseAdv_DNNhinge'+str(i)+'2'+str(j)+str(k)+'.out','r')
            best_val_acc = 0.0
            best_test_acc = 0.0
            for line in f:
                if line.startswith('accuracy is'): 
                    try:
                        val_acc = float(line.split(' ')[-2])
                    except:
                        print line
                        val_acc = float(line.split(' ')[-1])
                    if val_acc >= best_val_acc:
                        best_val_acc = val_acc
                if line.startswith('test set accuracy'):
                    test_acc = float(line.split(' ')[-2])
                    if test_acc >= best_test_acc:
                        best_test_acc = test_acc
            best_val_acc_ls[i][j][k] = best_val_acc
            best_test_acc_ls[i][j][k] = best_test_acc
            f.close()

accuracy is:  56.End - Sun Dec 11 05:21:10 CST 2016



In [10]:
print best_val_acc_ls
print best_test_acc_ls

[[[   56.81453768    57.29556387]
  [   57.18866916    57.13522181]]

 [[   57.34901122    57.616248  ]
  [   56.70764297    57.08177445]]

 [[   57.08177445  2016.        ]
  [   56.81453768    57.24211652]]]
[[[ 55.69214324  56.01282736]
  [ 55.53180118  55.95938001]]

 [[ 56.28006414  56.54730091]
  [ 55.90593266  55.74559059]]

 [[ 55.90593266  55.31801176]
  [ 55.90593266  55.90593266]]]


In [21]:
import theano
import theano.tensor as T

a = T.matrix()

b = T.argsort(a, axis = 1)

middle = T.arange(a.shape[0]).dimshuffle(0,'x')

c = T.argmin(T.abs_(b - middle), axis = 1)

test = theano.function([a], c)

In [22]:
import numpy as np

a_t = np.random.randint(20, size=(5,5))
arg_list = test(a_t)
print a_t
print arg_list

[[12 12  4 13 11]
 [16 13  0  9  7]
 [11  2  3  4  3]
 [10 15 17 13 10]
 [ 4  4  3  7  4]]
[2 3 1 2 3]


In [4]:
print np.__version__

1.11.0


In [39]:
import DNN_score_function
import lasagne

class model():
    def __init__(self):
        self.a = T.matrix()
        self.b = T.matrix()
        self.batchsize = self.a.shape[0]
        self.sent_rnn_units = [self.a.shape[1]]

        self.DNN_score_func = DNN_score_function.DNN(3,[5,3])
        self.c = self.matrix_cos(self.a, self.b)
        self.test = theano.function([self.a,self.b], self.c)
        
    def matrix_DNN(self, batch_rep1, batch_rep2):
        batch_rep1_broad = batch_rep1 + T.zeros((self.batchsize, self.batchsize, self.sent_rnn_units[-1]))

        batch_rep2_broad = batch_rep2 + T.zeros((self.batchsize, self.batchsize, self.sent_rnn_units[-1]))
        batch_rep1_reshape = batch_rep1_broad.dimshuffle(1,0,2).reshape((-1, self.sent_rnn_units[-1]))
        batch_rep2_reshape = batch_rep2_broad.reshape((-1, self.sent_rnn_units[-1]))

        batch_concate_input = T.concatenate([batch_rep1_reshape, batch_rep2_reshape], axis = 1)
        batch_score = lasagne.layers.get_output(self.DNN_score_func.output,
                                               {self.DNN_score_func.l_in: batch_concate_input})
        return batch_score.reshape((self.batchsize, self.batchsize))

    def matrix_cos(self, batch_rep1, batch_rep2):
        batch_rep1_broad = batch_rep1 + T.zeros((self.batchsize, self.batchsize, self.sent_rnn_units[-1]))

        batch_rep2_broad = batch_rep2 + T.zeros((self.batchsize, self.batchsize, self.sent_rnn_units[-1]))
        batch_rep1_reshape = batch_rep1_broad.dimshuffle(1,0,2).reshape((-1, self.sent_rnn_units[-1]))
        batch_rep2_reshape = batch_rep2_broad.reshape((-1, self.sent_rnn_units[-1]))

        batch_dot = (T.batched_dot(batch_rep1_reshape, batch_rep2_reshape)).reshape((self.batchsize, self.batchsize))
        norm1 = T.sqrt(T.sum(T.sqr(batch_rep1), axis = 1))
        norm2 = T.sqrt(T.sum(T.sqr(batch_rep2), axis = 1))

        norm_matrix = T.dot(norm1.reshape((-1,1)),norm2.reshape((1,-1)))
        return batch_dot/norm_matrix


In [41]:
model_t = model()

a = np.arange(3*3).reshape(3,3)
b = np.arange(9,18).reshape(3,3)
test_result = model_t.test(a,b)
print a
print b
print test_result

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[ 9 10 11]
 [12 13 14]
 [15 16 17]]
[[  32.   41.   50.]
 [ 122.  158.  194.]
 [ 212.  275.  338.]]


In [47]:
a = T.vector()
b = T.vector()
c = T.dot(a.reshape((-1,1)), b.reshape((1,-1)))

test = theano.function([a,b],c)

In [48]:
a_t = np.arange(3)
b_t = np.asarray([5,7,9])
c_t = test(a_t, b_t)

In [50]:
print a_t
print b_t
print c_t


[0 1 2]
[5 7 9]
[[  0.   0.   0.]
 [  5.   7.   9.]
 [ 10.  14.  18.]]


In [51]:
pwd

u'/home-nfs/jontsai/ROC/DSSM/adversarial'

In [53]:
import cPickle as pickle

file_handle = pickle.load(open('../../data/pickles/adv_mean_std_record.pkl','r'))

print len(file_handle)

4


In [8]:
import lasagne

import theano
import theano.tensor as T
import LSTM_Decoder

De = LSTM_Decoder.LSTMDecoder([5,3,2])
De.index_label_classi_decoder()

a = T.matrix(dtype=theano.config.floatX)
a_in = a.dimshuffle(0,1,'x') + T.zeros((a.shape[0], a.shape[1], 5))
a_mask = T.ones_like(a)
b = lasagne.layers.get_output(De.output, {De.l_in: a_in, De.l_mask: a_mask})

test = theano.function([a], b)





In [9]:
import numpy as np

a_t = np.arange(5*7).reshape((5,7))

test(a_t)

array([[[ 0.1202812 ,  0.12295712],
        [ 0.21835778,  0.18777873],
        [ 0.29210087,  0.23609609],
        [ 0.3339535 ,  0.26655077],
        [ 0.35441106,  0.291316  ],
        [ 0.36623037,  0.3151724 ],
        [ 0.37558322,  0.33907216]],

       [[ 0.16643457,  0.11393597],
        [ 0.28953943,  0.18735294],
        [ 0.3531192 ,  0.23552933],
        [ 0.38666388,  0.27086792],
        [ 0.40723356,  0.29948229],
        [ 0.42223229,  0.32449283],
        [ 0.43486659,  0.34739558]],

       [[ 0.17443445,  0.11033654],
        [ 0.30764507,  0.18016743],
        [ 0.37960195,  0.22281068],
        [ 0.41899273,  0.25193557],
        [ 0.44344687,  0.27428244],
        [ 0.46097165,  0.29310206],
        [ 0.47521528,  0.30996493]],

       [[ 0.1771425 ,  0.10942381],
        [ 0.31325719,  0.17823762],
        [ 0.38818835,  0.21918645],
        [ 0.42998985,  0.24614957],
        [ 0.45637298,  0.26599533],
        [ 0.47552038,  0.28201028],
        [ 0.49115542, 

In [28]:
a = T.arange(5).dimshuffle('x',0) + T.zeros((5,5))
b = a.dimshuffle(0,1,'x') + T.zeros((5,5,1))


c = T.arange(5*5*3).reshape((5,5,3))

d = T.concatenate([b,c], axis = 2)
test = theano.function([], d)


In [29]:
test()

array([[[  0.,   0.,   1.,   2.],
        [  1.,   3.,   4.,   5.],
        [  2.,   6.,   7.,   8.],
        [  3.,   9.,  10.,  11.],
        [  4.,  12.,  13.,  14.]],

       [[  0.,  15.,  16.,  17.],
        [  1.,  18.,  19.,  20.],
        [  2.,  21.,  22.,  23.],
        [  3.,  24.,  25.,  26.],
        [  4.,  27.,  28.,  29.]],

       [[  0.,  30.,  31.,  32.],
        [  1.,  33.,  34.,  35.],
        [  2.,  36.,  37.,  38.],
        [  3.,  39.,  40.,  41.],
        [  4.,  42.,  43.,  44.]],

       [[  0.,  45.,  46.,  47.],
        [  1.,  48.,  49.,  50.],
        [  2.,  51.,  52.,  53.],
        [  3.,  54.,  55.,  56.],
        [  4.,  57.,  58.,  59.]],

       [[  0.,  60.,  61.,  62.],
        [  1.,  63.,  64.,  65.],
        [  2.,  66.,  67.,  68.],
        [  3.,  69.,  70.,  71.],
        [  4.,  72.,  73.,  74.]]])